### This is just an example of how current version can be used

In [ ]:
from synaptics import * # Synaptics includes import of layers and numpy
from visual_env import *
from stats import *

#### Let's build our model
In current version, the model as a single object is not implemented, so we basically create layers of neurons and synapses.
<br/>We will use a frame 5 by 5 as visual environment, and the object is a oblique line that moves in two directions (left-right, up-down, doesn't really matter because of it's symmetry)

In [ ]:
frame_size = 5,5 # define the frame size

input_size = frame_size[0] * frame_size[1] # area of the frame is equal to amount of input neurons
output_size = 1 # the model will have only one neuron in this case. It should fire when familiar pattern is spotted
resolution = .1 # this parameter describes how often neural dynamics data is updated
preset = 'RS' # the type of neuron's dynamics from Izhikevich paper - https://ieeexplore.ieee.org/document/1257420
tau = 30 # characteristic time of STDP. In current version, this variable is integrated in neuron's model. Later it probably will be moved to Synapse

# Creating layers themselves, we will have two of them:
input_layer = IzhikevichLayer(size=input_size, resolution=resolution, tau=tau,
                              preset=preset, noize=1)
output_layer = IzhikevichLayer(size=output_size, resolution=resolution, tau=tau,
                               preset=preset, noize=1)

# Creating connections between layers (e.g. synapses):
synapse = Synapse(input_layer, output_layer)

# We will set all weights to 1. Empirically, that is the best way to train such model:
synapse.weights *= 0
synapse.weights += 1

print(synapse.weights.T)

g = 47 # this parameter scales the impact of presynaptic action potentials to postsynaptic activation
input_layer.transmitter_impact = g
print(synapse.weights.reshape(5,5))

Our moving pattern will be an oblique line

In [ ]:
vis = motion_pic(size=frame_size) # Creating the visual field
pattern = np.eye(5)* 15
vis.add_object(pattern) # Adding the patern
vis.set_position_lazy(x='centered', y='centered')
print(f'This is how our pattern looks like: \n', vis.show_current_state())

In [ ]:
vis.set_position_lazy(x='left', y='centered') # let's say, we aregoing to move our pattern from left to the right

In [ ]:
# these objects will help us to estimate firing rates:
rates0 = rate_capture(input_layer)
rates1 = rate_capture(output_layer)

In [ ]:
# Notice that hyperparameters are not at optimal value. Tuning them is in my to-do list
# As well as annotating the rest of the code
def train_model(t, vis, rates1, resolution=.1, delay=4,
                 gather_data = True, train=False, lr = .01, alpha = 5):
    time = int(t / resolution)
    pattern_delay = int(delay/ resolution)
    directions = 'right',
    positions = 'left',
    v1_right = []
    v1_left = []
    v0_right = []
    v0_left = []
    weights_hist = []
    direction = directions[0]
    position = positions[0]

    vis.set_position_lazy(x='left', y='centered')
    print(f'Moving pattern to the {direction}')
    for j in range(time):
        synapse.forward()
        picture = vis.tick(delay=pattern_delay, move_direction=direction, noise_density=.1, noise_acceleration=5, rest=int(120/resolution))
        input_layer.apply_current(picture.flatten())
        output_layer.forward()
        rates0.accumulate_spikes()
        rates1.accumulate_spikes()
        if train:
            synapse.STDP(learning_rate=lr, assymetry=alpha)
        if gather_data:
            if direction == 'right':
                v0_right.append(input_layer.v.tolist())
                v1_right.append(output_layer.v.item())
            if direction == 'left':
                v0_left.append(input_layer.v.tolist())
                v1_left.append(output_layer.v.item())
            weights_hist.append(synapse.weights.tolist())
            
    input_layer.instant_rest()
    output_layer.instant_rest()
    r1 = rates1.compute_spike_rates(time=t, interval=1000)
    rates0.reset()
    rates1.reset()
    print(f'{position}-{direction} direction output rates', r1)
    print('Completed!')
    return v0_right, v0_left, v1_right, v1_left, weights_hist

In [ ]:
v0_right, v0_left, v1_right, v1_left, weights_hist = train_model(t=10000, vis=vis, rates1=rates1, train=True)
weights_hist = np.array(weights_hist)
v1_right = np.array(v1_right)
v1_left = np.array(v1_left)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(v0_right)
plt.show()
plt.plot(v1_right)
plt.show()
for i in range(input_size):
    plt.plot(weights_hist[:,i])
plt.show()

In [ ]:
def test(t, vis, rates1, num_attempts=15, resolution=.1, delay=4):
    time = int(t / resolution)
    pattern_delay = int(delay/ resolution)
    directions = 'right', 'left'
    positions = 'left', 'right'
    r_right = []
    r_left = []

    for d in range(len(directions)):
        direction = directions[d]
        position = positions[d]
        for i in range(num_attempts):
            vis.set_position_lazy(x=position, y='centered')
            print(f'Moving pattern to the {direction}: attempt num {i}')
            for j in range(time):
                synapse.forward()
                picture = vis.tick(delay=pattern_delay, move_direction=direction, noise_density=.1, noise_acceleration=5, rest=int(120/resolution))
                input_layer.apply_current(picture.flatten())
                output_layer.forward()
                rates0.accumulate_spikes()
                rates1.accumulate_spikes()
            input_layer.instant_rest()
            output_layer.instant_rest()
            r1 = rates1.compute_spike_rates(time=t, interval=1000)
            rates0.reset()
            rates1.reset()
            if direction == 'right':
                r_right.append(r1.item())
            else:
                r_left.append(r1.item())
    print('Completed!')
    return r_right, r_left

In [ ]:
input_layer.transmitter_impact = 47
r_right, r_left = test(t=10000, vis=vis, rates1=rates1, num_attempts=15)

In [ ]:
print(r_right)
print(r_left)